<a href="https://colab.research.google.com/github/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/CE888_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        #print("st in Clone function : ",st.board) # TEST PRINT
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        #print("move value in Do Move function before assert : ", move, "and self board : ", self.board)
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0 # the random slection is between 0 and 8 (inclusive) and the board has that index as 0 and the integer value of passed random number is same as expected
        self.playerJustMoved = 3 - self.playerJustMoved # 3-1 because everytime player 1 plays after player 2, 3-1 gives the value. Set the value as player 1, so far it was assumed player 1 is playing but 2 was assigned
        print("\n In DoMove, updated player just moved in self : ", self.playerJustMoved)
        self.board[move] = self.playerJustMoved # it is the player that is playing for now, assign player 1 to the random index of the board
        print("\n In DoMove, updated self.board[move] in self, printing the board : ", self.board)
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0] #returns the list of playable positions in the current state. that is if it is 0.
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]: #winning possibiliteis
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm: #if the value on the state board is 1 and the pjm is also one then return 1. Similarly for 2
                    return 1.0
                else: # the other player wins
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        assert False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        #print("OXOState return : ", s)
        return s
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        print("In Addchild")
        #print("In AddChild : print self : ", self)
        n = Node(move = m, parent = self, state = s)
        #print("In AddChild : print n : ", n)
        #print("In AddChild, n created but parent value of self is  : ", self.parentNode, " SELF : ", self)
        print("In AddChild, n created but parent value of n is  : ", n.parentNode, " n : ", n)
        #print("In AddChild, n created but child value of self is  : ", self.childNodes)
        #print("In AddChild, n created but child value of n is  : ", n.childNodes)
        self.untriedMoves.remove(m)
        #print("In AddChild, after removing untried moves from self, print self : ", self)
        #print("In AddChild, after removing untried moves from self, print n : ", n)
        #print("childnodes list : ", self.childNodes)
        self.childNodes.append(n)
        #print("In AddChild, after append childnodes to self- inpur is n- , print self : ", self.childNodes)
        #print("In AddChild, after append childnodes to self- inpur is n- , print n : ", n.childNodes)
        #print("childnodes list after append: ", self.childNodes, "\n n.childnodes", n.childNodes)
        #print("In AddChild, after append, print n : ", n)
        print("End of Addchild")
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + " Parent Node (repr) : " + str(self.parentNode) + "]"
        #print("Nodereturn : ", Nodereturn)
        #return Nodereturn

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False): #pass state as rootstate
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)
    
    #print(rootnode, "rotnode parent values : ", rootnode.parentNode, "type of parent node : ", type(rootnode.parentNode)) # TEST PRINT
    
    
    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone() #the initial state variable of OXOState typee is passed as rootstate, state here is local?

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            print("In while loop of fully expanded node.--------------------------------------------------")
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            print("in if condition : -------------------------------------------------------", node.untriedMoves)
            m = random.choice(node.untriedMoves) # random move value from the untried list. local m to this function
            print("random m decided : ", m)
            state.DoMove(m) # changed the boards #lets say random was decided as 5 then print [0,0,0,0,0,1,0,0,0], it is not yet decided if this is the final move
            node = node.AddChild(m,state) # add child and descend tree, local variables passed
            print("node:", node, "node parent node : ", node.parentNode, "node child nodes : ", node.childNodes, "node pjm : ", node.playerJustMoved)
        #print("player just moved",state.playerJustMoved)
        
        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #print("In rollback while : -------------------------------------------------------", state.GetMoves())
            x=random.choice(state.GetMoves())
            print("In rollout, random value chosen is : ", x)
            state.DoMove(x)
        #print("Outside Rollout.")
        #print("node n:",node)
        #print("player just moved",state.playerJustMoved)
        
        #print("\n Before backpropogate, values of node and state")
        print("\n NODE move : ", node.move, "node.parent : ", node.parentNode, "node.child : ", node.childNodes, "wins : ", node.wins, "visits : ", node.visits, "untried moves : ", node.untriedMoves, "pjm : ",node.playerJustMoved)

        print("\n STATE board : ", state.board, "pjm : ",state.playerJustMoved)

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            print("backpropogate while, print node : ", node, "node.playerjustmoved : ", node.playerJustMoved)
            y=state.GetResult(node.playerJustMoved)
            print("state.getresult of the node.player just moved : ", y)
            node.Update(y) # state is terminal. Update node with result from POV of node.playerJustMoved
            print("after updating node : ", node, "parents : ", node.parentNode, "child : ", node.childNodes, "pjm : ", node.playerJustMoved)
            #QUERY - how did pjm become 2?
            node = node.parentNode
        print("after backpropagation node : ", node)
    
    # Output some information about the tree - can be omitted
    if (verbose): print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())
    #print("childnodes before sorting return : ", rootnode.childNodes)
    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
                
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    
    stateUCT = OXOState()
    
    while (stateUCT.GetMoves() != []):
              
        #print(str(stateUCT)) # prints the matrix with dots or X or O #QUERY - how to get X and 0
        
        if stateUCT.playerJustMoved == 1:
            m = UCT(rootstate = stateUCT, itermax = 5, verbose = False) # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate = stateUCT, itermax = 5, verbose = False)
        print("Best Move: " + str(m) + "\n")
        stateUCT.DoMove(m)
        print("in UCTPLayGame after Do Move in while, state UCT is : ", str(stateUCT))
    
    if stateUCT.GetResult(stateUCT.playerJustMoved) == 1.0:
        print("Player " + str(stateUCT.playerJustMoved) + " wins!")
    elif stateUCT.GetResult(stateUCT.playerJustMoved) == 0.0:
        print("Player " + str(3 - stateUCT.playerJustMoved) + " wins!")
    else: print("Nobody wins!")
    ######append the bxest move from stateuct.board

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    UCTPlayGame()

in if condition : ------------------------------------------------------- [0, 1, 2, 3, 4, 5, 6, 7, 8]
random m decided :  3

 In DoMove, updated player just moved in self :  1

 In DoMove, updated self.board[move] in self, printing the board :  [0, 0, 0, 1, 0, 0, 0, 0, 0]
In Addchild
In AddChild, n created but parent value of n is  :  [M:None W/V:0/0 U:[0, 1, 2, 3, 4, 5, 6, 7, 8] Parent Node (repr) : None]  n :  [M:3 W/V:0/0 U:[0, 1, 2, 4, 5, 6, 7, 8] Parent Node (repr) : [M:None W/V:0/0 U:[0, 1, 2, 3, 4, 5, 6, 7, 8] Parent Node (repr) : None]]
End of Addchild
node: [M:3 W/V:0/0 U:[0, 1, 2, 4, 5, 6, 7, 8] Parent Node (repr) : [M:None W/V:0/0 U:[0, 1, 2, 4, 5, 6, 7, 8] Parent Node (repr) : None]] node parent node :  [M:None W/V:0/0 U:[0, 1, 2, 4, 5, 6, 7, 8] Parent Node (repr) : None] node child nodes :  [] node pjm :  1
In rollout, random value chosen is :  0

 In DoMove, updated player just moved in self :  2

 In DoMove, updated self.board[move] in self, printing the board :  [2, 0, 

In [52]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []:
          return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s


class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose: print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
                
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    state = OXOState() # uncomment to play OXO

    tempdataset=[[0,0,0,0,0,0,0,0,0]] #LINE ADDED
    bestmovelist=[] # LINE ADDED
    while state.GetMoves() != []:
        print(str(state))
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        print("Best Move: " + str(m) + "\n")
        state.DoMove(m)
        bestmovelist.append((m)) # LINE ADDED
        tempdataset.append((state.board))  #LINE ADDED
        #print(type(state.board))
        if state.GetResult(state.playerJustMoved) != False:
            print(str(state))            
            break

    if state.GetResult(state.playerJustMoved) == 1.0:
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else: print("Nobody wins!")

    bestmovelist.append('NaN')  # LINE ADDED
    print("DATASETS : ") # LINE ADDED

    for i in range(len(tempdataset)): # LINE ADDED
      #print(tempdataset[i] + bestmovelist[i]) # LINE ADDED
      #tempdataset[i] = tempdataset[i] + ',' + bestmovelist[i] # LINE ADDED
      print(tempdataset[i]) # LINE ADDED
    print("\n") # LINE ADDED
    for i in bestmovelist: # LINE ADDED
      print(i) # LINE ADDED

    return tempdataset # LINE ADDED

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    finaldataset=[] # LINE ADDED
    finaldataset1=[] # LINE ADDED
    for i in range(2): # LINE ADDED
      returnds=UCTPlayGame() # LINE EDITED
      finaldataset.append(returnds) # LINE ADDED
    print("\n FINALDATASET") # LINE ADDED
    for i in finaldataset: # LINE ADDED
      print(i) # LINE ADDED
    #Convert list of lists to one list # LINE ADDED
    print("\n FINALDATASET1") # LINE ADDED
    
    for i in finaldataset:  # LINE ADDED
      for j in i:  # LINE ADDED
        finaldataset1.append(j)  # LINE ADDED
    for i in finaldataset1: # LINE ADDED
      print(i) # LINE ADDED

    #hmmmmmmmmmmm

    #("hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh")
    #for i in range(len(finaldataset1)):
    #  for j in range(len(finaldataset1[0])):
    #    if (finaldataset1[i][j] in ('[',']',',')):
          #print(finaldataset1[i][j])
          #del(finaldataset1[i][j])
    #      finaldataset1[i][j].remove()
    #for i in (finaldataset1):
    #  finaldataset1.translate({ord(i): None for i in '[],'})
    #print(finaldataset1[0][0])

...
...
...

[M:4 W/V:8.5/12 U:[]]
[M:0 W/V:10.5/14 U:[]]
[M:2 W/V:10.0/13 U:[]]
[M:1 W/V:11.5/14 U:[]]
[M:7 W/V:2.0/6 U:[0, 3, 6]]
[M:6 W/V:9.5/13 U:[]]
[M:3 W/V:4.5/8 U:[8]]
[M:5 W/V:2.5/7 U:[7, 8]]
[M:8 W/V:10.0/13 U:[]]

Best Move: 1

.X.
...
...

[M:3 W/V:24.5/88 U:[]]
[M:0 W/V:146.0/314 U:[]]
[M:2 W/V:14.0/66 U:[]]
[M:4 W/V:110.5/251 U:[]]
[M:5 W/V:9.5/55 U:[]]
[M:6 W/V:31.0/102 U:[]]
[M:7 W/V:13.0/63 U:[]]
[M:8 W/V:12.0/61 U:[]]

Best Move: 0

OX.
...
...

[M:5 W/V:15.0/21 U:[]]
[M:2 W/V:3.0/9 U:[]]
[M:6 W/V:4.0/10 U:[]]
[M:3 W/V:6.0/12 U:[]]
[M:7 W/V:3.5/10 U:[]]
[M:4 W/V:8.5/15 U:[]]
[M:8 W/V:17.0/23 U:[]]

Best Move: 8

OX.
...
..X

[M:6 W/V:224.0/399 U:[]]
[M:2 W/V:36.5/99 U:[]]
[M:5 W/V:56.5/133 U:[]]
[M:3 W/V:52.0/126 U:[]]
[M:7 W/V:30.5/88 U:[]]
[M:4 W/V:69.0/155 U:[]]

Best Move: 6

OX.
...
O.X

[M:5 W/V:4.0/15 U:[]]
[M:3 W/V:30.0/47 U:[]]
[M:4 W/V:3.5/14 U:[]]
[M:7 W/V:1.5/11 U:[]]
[M:2 W/V:3.0/13 U:[]]

Best Move: 3

OX.
X..
O.X

[M:5 W/V:120.0/269 U:[]]
[M:4 W/V:327.0

In [46]:
print(len(finaldataset1[0]))
print(finaldataset1[0])
finaldataset1[0].translate({ord('i'):None for i in '[],'})
print(finaldataset1[0])

29
[0, 0, 0, 0, 0, 0, 0, 0, 0],4
[0, 0, 0, 0, 0, 0, 0, 0, 0],4
